In [1]:
from absl import app, flags
from acme.agents.tf import d4pg

2021-08-18 17:43:10.588325: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /v/apps/clang/8.0.1/lib:/v/apps/fftw/3.3.7/gcc_8.2/lib:/v/apps/cuda/10.1.168/lib64:/v/apps/mpi/mvapich2/2.3/gcc_8.2/lib:/v/apps/gcc/8.2/lib64:/v/apps/valgrind/3.12.0/lib/valgrind:/v/apps/gdb/7.12-centos7/lib64::/u/samar/.mujoco/mujoco200/bin:/u/samar/Documents/git/protobuf/protbuf_install:/u/samar/Documents/git/caffe2/install/lib:/u/samar/Documents/git/protobuf/protbuf_install/:/u/samar/Documents/git/caffe2/install/lib:/u/samar/.conda/envs/rl/lib
2021-08-18 17:43:10.588365: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import acme
from acme import specs, wrappers
from acme.agents.tf import dqn

In [3]:
import bsuite
import sonnet as snt


In [4]:
# Bsuite flags
flags.DEFINE_string('bsuite_id', 'deep_sea/0', 'Bsuite id.')
flags.DEFINE_string('results_dir', '/tmp/bsuite', 'CSV results directory.')
flags.DEFINE_boolean('overwrite', False, 'Whether to overwrite csv results.')
FLAGS = flags.FLAGS

In [6]:
import copy
import pyvirtualdisplay
import imageio 
import base64
import IPython


from acme import environment_loop
from acme.tf import networks
from acme.adders import reverb as adders
from acme.agents.tf import actors as actors
from acme.datasets import reverb as datasets
from acme.wrappers import gym_wrapper
from acme import specs
from acme import wrappers
from acme.agents.tf import d4pg
from acme.agents import agent
from acme.tf import utils as tf2_utils
from acme.utils import loggers

import gym 
import dm_env
import matplotlib.pyplot as plt
import numpy as np
import reverb
import sonnet as snt
import tensorflow as tf

# Import dm_control if it exists.
try:
  from dm_control import suite
except (ModuleNotFoundError, OSError):
  pass

# Set up a virtual display for rendering OpenAI gym environments.
#display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [10]:
#environment_name = 'dm_cartpole'  # @param ['dm_cartpole', 'gym_mountaincar']
environment_name = 'gym_mountaincar'

# task_name = 'balance'  # @param ['swingup', 'balance']

def make_environment(domain_name='cartpole', task='balance'):
  env = suite.load(domain_name, task)
  env = wrappers.SinglePrecisionWrapper(env)
  return env

if 'dm_cartpole' in environment_name:
  environment = make_environment('cartpole')
  def render(env):
    return env._physics.render(camera_id=0)  #pylint: disable=protected-access

elif 'gym_mountaincar' in environment_name:
  environment = gym_wrapper.GymWrapper(gym.make('MountainCarContinuous-v0'))
  environment = wrappers.SinglePrecisionWrapper(environment)
  def render(env):
    return env.environment.render(mode='rgb_array')
else:
  raise ValueError('Unknown environment: {}.'.format(environment_name))

# Show the frame.
#frame = render(environment)
#plt.imshow(frame)
#plt.axis('off')

In [11]:
environment